# Simulation of the Process with Process Variations

Import PyRolL modules.

In [ ]:
import pyroll.basic as pr
import pyroll.export as pre

Import the process data.

In [ ]:
from weiner_variation.sim.process import PASS_SEQUENCE, IN_PROFILE, TEMPERATURE, DIAMETER, TEMPERATURE_STD, DIAMETER_STD, create_in_profile
from weiner_variation.sim.config import SAMPLE_COUNT, SEED
from weiner_variation.sim.data import DrawDurations
from weiner_variation.config import DATA_DIR

Import further libraries.

In [ ]:
import pandas as pd
from copy import deepcopy
from scipy.stats import norm, weibull_min
import numpy as np
import tqdm
from multiprocessing import Pool

Parameters for notebook using papermill.

In [ ]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = DATA_DIR / "sim_durations_results.csv"

Load pause duration data in a dataframe.

In [ ]:
df_durations = pd.read_csv(DATA_DIR / "duo_pauses_dist.csv", header=0, index_col=0)
df_durations

Create distribution functions for input values.

In [ ]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)
durations_dists = [weibull_min(c=r["shape"], scale=r["scale"]) for i, r in df_durations.iterrows()]

Draw random inputs from distributions.

In [ ]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
durations = np.concatenate([
    d.rvs(random_state=RNG, size=(SAMPLE_COUNT, 1))
    for d in durations_dists[:-1]
], axis=1)

draws = [
    DrawDurations(d, t, dur)
    for d, t, dur in zip(diameters, temperatures, durations)
]

Define a worker function creating the in profile, running the solution procedure and extracting results.

In [ ]:
def worker(draw: DrawDurations):    
    ip = create_in_profile(draw.diameter)
    ip.temperature = draw.temperature

    sequence = deepcopy(PASS_SEQUENCE)  
    
    transports = [u for u in sequence if isinstance(u, pr.Transport)]
    for t, d in zip(transports, draw.durations):
        t.duration = d
    
    sequence.solve(ip)       

    return pre.to_pandas(sequence)

Run the simulations using a process pool.

In [ ]:
results = list(tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT))

Create a dataframe from results.

In [ ]:
df = pd.DataFrame([r.stack().swaplevel().sort_index() for r in results])
df

Save dataframe to CSV file.

In [ ]:
df.to_csv(OUTPUT_FILENAME)